아래는 딥러닝 기반의 얼굴 인식 코드 인데 파일 경로 오류 발생 

In [ ]:
import cv2
import numpy as np
import pyautogui
import time

# 1. 딥러닝 모델 로드 (Haar Cascade보다 훨씬 강력함)
model_path = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
config_path = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(config_path, model_path)

# 파란색 L자 영역 정의
ROI_POINTS_NORM = np.array([
    [0.10, 0.05], [0.90, 0.05], [0.90, 0.45], 
    [0.35, 0.45], [0.35, 1.00], [0.10, 1.00]
], dtype=np.float32)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

last_action_time = 0
print("딥러닝 기반 원거리 감시 시작...")

while True:
    ret, frame = cap.read()
    if not ret: break

    h, w = frame.shape[:2]
    roi_pts = (ROI_POINTS_NORM * [w, h]).astype(np.int32)
    
    # 2. 딥러닝 입력을 위한 전처리 (Blob 생성)
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    detected_in_target = False

    # 3. 감지 결과 분석
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        # 신뢰도가 낮아도(0.3 이상) 일단 감지하도록 설정 (원거리 대응)
        if confidence > 0.3:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # 감지된 위치가 파란색 ROI 내부인지 확인
            if cv2.pointPolygonTest(roi_pts, (float(cx), float(cy)), False) >= 0:
                detected_in_target = True
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{confidence:.2f}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 4. 실행 (Win+D)
    now = time.time()
    if detected_in_target and (now - last_action_time) > 5:
        pyautogui.hotkey("win", "d")
        last_action_time = now

    cv2.polylines(frame, [roi_pts], True, (255, 0, 0), 2)
    cv2.imshow("High-Precision Guard", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

아래는 움직임 감지 코드

영역 조절: ROI_POINTS_NORM의 첫 번째 열(0.28, 0.36 등)을 수정하여 세로 통로의 너비를 조절할 수 있습니다. 현재는 사진 속 빨간색 사각형처럼 왼쪽 벽면을 살짝 띄운 통로 구간을 조준하고 있습니다.

민감도 튜닝: cv2.contourArea(contour) < 1500

파티션 너머 멀리서 오는 사람을 더 빨리 잡으려면 이 값을 **800~1000**으로 낮추세요.

너무 예민해서 혼자 꺼진다면 2000 이상으로 높이시면 됩니다.

배경 갱신(R키): 사무실 조명이 바뀌거나 의자 위치가 바뀌어 계속 오작동할 때 'R' 키를 한 번 눌러주면 현재 상태를 깨끗한 배경으로 다시 저장합니다

In [ ]:
import cv2
import numpy as np
import ctypes
import time

# 윈도우 키 입력을 위한 시스템 상수 및 함수
user32 = ctypes.windll.user32
def instant_minimize():
    # Win + M 신호를 시스템 레벨에서 즉시 발생 (토글 현상 없음, 매우 빠름)
    user32.keybd_event(0x5B, 0, 0, 0) # Left Windows key down
    user32.keybd_event(0x4D, 0, 0, 0) # M key down
    user32.keybd_event(0x4D, 0, 2, 0) # M key up
    user32.keybd_event(0x5B, 0, 2, 0) # Left Windows key up

# 박사님의 파티션 밀착 ROI 좌표
ROI_POINTS_NORM = np.array([
    [0.25, 0.08], [0.85, 0.08], [0.85, 0.44], 
    [0.38, 0.44], [0.38, 0.7], [0.25, 0.7]
], dtype=np.float32)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# 카메라 안정화 (버퍼 비우기)
for _ in range(5): cap.read()

static_back = None
last_action_time = 0

print("초고속 보안 감시 모드 가동 중... (Q: 종료)")

while True:
    ret, frame = cap.read()
    if not ret: break

    h, w = frame.shape[:2]
    roi_pts = (ROI_POINTS_NORM * [w, h]).astype(np.int32)
    
    # 전처리 속도 최적화 (가우시안 블러 크기 축소 21->11)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)

    if static_back is None:
        static_back = gray
        continue

    # 배경 차분 및 이진화 (속도 위주 설정)
    diff_frame = cv2.absdiff(static_back, gray)
    thresh_frame = cv2.threshold(diff_frame, 35, 255, cv2.THRESH_BINARY)[1]

    # ROI 영역 추출
    mask = np.zeros_like(thresh_frame)
    cv2.fillPoly(mask, [roi_pts], 255)
    roi_motion = cv2.bitwise_and(thresh_frame, thresh_frame, mask=mask)

    # 움직임 감지 (컨투어 연산을 생략하고 픽셀 합계로 속도 극대화)
    motion_score = np.sum(roi_motion)
    
    now = time.time()
    # 일정량 이상의 픽셀 변화가 생기면 즉시 실행 (기준값 500,000은 환경에 따라 조절)
    if motion_score > 500000 and (now - last_action_time) > 2:
        instant_minimize() # 가장 빠른 최소화 명령
        print(f"!!! 즉각 대응 실행: {time.strftime('%H:%M:%S')} !!!")
        last_action_time = now
        static_back = gray # 현재 프레임을 새 배경으로

    # 모니터링 출력
    cv2.polylines(frame, [roi_pts], True, (0, 0, 255), 3)
    cv2.imshow("Fast Guard", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    if key == ord('r'): static_back = None

cap.release()
cv2.destroyAllWindows()

초고속 보안 감시 모드 가동 중... (Q: 종료)
!!! 즉각 대응 실행: 13:12:31 !!!
!!! 즉각 대응 실행: 13:12:33 !!!
!!! 즉각 대응 실행: 13:12:36 !!!
!!! 즉각 대응 실행: 13:14:24 !!!
!!! 즉각 대응 실행: 13:14:26 !!!
